# **Swyp Teddy Bear Club NFT Rating and Rewards Calculation**

## **1. Overview**

### **1.1 Purpose**
This document outlines the process for calculating the rankings and rewards for the Teddy Bear Club NFTs across Round 1 and Round 2. Since the ranking and reward structures differ between the two rounds, calculations are performed separately for each round and then added to determine the total rewards, where applicable.

### **1.2 NFT Information**
Round 1:<br>
   - Policy Id: ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87 <br>
   - Total NFTs: 804<br>
   - Tier System:<br>
      - Tier 1 - Rarity 1 - 15 
      - Tier 2 - Rarity 16 - 100 
      - Tier 3 - Rarity 101 - 250 
      - Tier 4 - Rarity 251 - 500 
      - Tier 5 - Rarity 501 - 804  
      

Round 2:<br>
   - Policy Id: da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c<br>
   - Total NFTs: 2,035<br>
   - Tier System:<br>
      - Tier 1 - Rarity 1 - 23 
      - Tier 2 - Rarity 24 - 221 
      - Tier 3 - Rarity 222 - 664
      - Tier 4 - Rarity 665 - 1,327 
      - Tier 5 - Rarity 1,328 - 2,035 

---

### **1.3 Workflow Summary**
The workflow is divided into the following steps:

1. **Fetch Data**  
   - Load NFT data from JSON files and rank data from the CSV file.

2. **NFT Rank Calculation**  
   2.1 Assign point values based on trait rarity.  
   2.2 Compute overall rarity ranks for each NFT.

3. **NFT Rewards Calculation**  
   3.1 Count the number of occurrences of each NFT within its respective tier.<br>
   3.2 Calculate the user’s percentage share across all sold NFTs.<br>
   3.3 Determine the user’s adjusted percentage share based on the remaining available Tedy token rewards.<br>
   3.4 Compute the total Tedy tokens using the number and rank of owned NFTs, factoring in the user’s percentage share.<br>
   3.5 Adjust the final Tedy token rewards according to the updated market cap of 8,000,000.<br>
   
---

### **1.4 Necessary Package Installations and Open Statements**


In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

#r "nuget: Plotly.NET.Interactive, 5.0.0"

In [68]:
open System.Text.Json
open System.IO
open FSharp.Reflection
open Plotly.NET
open Plotly.NET.Interactive

# **2. Workflow**

## 1. Fetch Data
The metadata for the NFTs was scraped from the web and stored in JSON files. In this section, we will parse the JSON data to extract information about the NFTs and the occurrence of their traits. This data will serve as the foundation for calculating rankings and determining rewards in the subsequent sections.

---

#### **Types**:
The NFT and NFTTraits types are partial representations of the data, as only the fields relevant to ranking and reward calculations have been included.

- **`NFT`**: Represents an individual NFT, including its metadata and traits.

- **`NFTTraits`**: Captures occurrence counts of trait values across NFTs.

#### **Function**:
1. **`loadNFTs<'T>`**: 
- Loads and deserializes JSON data into a specified type:
- **Parameters**: 
     - `filePath (string)`: the path to the JSON file.
- **Returns**: 
        - Parsed data of type `'T`

2. **`loadNFTRanks`**: 
- Loads and parses CSV data:
- **Parameters**:
    - `filePath (string)`: the path to the CSV file.
- **Returns**:  <br>
    - A (Map<string, int>) of NFT IDs to their ranks

--- 

#### **Variables**:<br>
**`nftRound1NFTS`**: <br>
- (NFT list): NFTs from Round 1 <br>

 **`nftRound2NFTS`**:  <br>
- (NFT list):  NFTs from Round 2 <br>

 **`nftRound1Traits`**: <br>
- (NFTTraits): Trait counts for Round 1 <br>
    
 **`nftRound2Traits`**:  <br>
- (NFTTraits): Trait counts for Round 2 <br>

<br>

**`subjectRankData`**: 
- (Map<string, int>): Mapping of NFTs to their calculated ranks

---
### 1.1 Fetch NFT Data From JSON

In [69]:
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NFTTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [70]:
let loadNFTs<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [71]:
let nftRound1NFTS = loadNFTs<NFT list> "./NFTR1.json"
let nftRound2NFTS = loadNFTs<NFT list> "./NFTR2.json"
let nftRound1Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT1Traits.json"
let nftRound2Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT2Traits.json"
let nftRound1Traits = List.head nftRound1Data
let nftRound2Traits = List.head nftRound2Data

### 1.2 Fetch Rank Data From CSV

In [72]:
let loadNFTRanks (filePath: string) =
    File.ReadAllLines(filePath)
    |> Array.skip 1
    |> Array.map (fun line ->
        let parts = line.Split(',')
        let subject = parts.[0]
        let rank = int parts.[1]
        subject, rank
    )
    |> Map.ofArray

In [73]:
let subjectRankData = loadNFTRanks "NFT_Ranks.csv"

## **2. NFT Rank Calculation** 
To calculate the points for each trait, we first divide the number of occurrences of that trait by the total number of NFTs in the round, which gives us the percentage of NFTs that have that trait. We then subtract this percentage from 100, creating a direct relationship between rarity and points so rarer traits get higher points. Finally, we sum the points for each trait across different attributes (such as Background, Bear, Clothes, etc.) to calculate the final rarity score for each NFT.

$\text{Trait Score} = \text{round}\left(100.0 - \left(\frac{\text{Trait Frequency}}{\text{Total NFTs}}\right) \cdot 100.0, 2\right)$

$\text{Total Rarity Score} = \sum_{i=1}^{7} T_i$

---
### 2.1 Assign point values based on trait rarity.   

#### **Function**:
1. **processNFTTraits**: 
- Calculates rarity scores for each trait dynamically for a given NFTTraits record and total NFTs in the round.
- **Parameters**:<br>
    - **`nftTraits`**: NFTTraits – The trait occurrence data for a round.
        
    - **`totalNfts`**: float – The total number of NFTs in the round.
- **Returns**: <br>
    - A Map<string, Map<string, float>> of categories, their traits, and the trait's number of occurrences

#### **Variables**:
**`r1TotalNfts`**: 
- (float): The total number of NFTs in Round 1 (804).<br>

**`r2TotalNfts`**: 
- (float): The total number of NFTs in Round 2 (2035).<br>

**`r1ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait percentages for Round 1 NFTs.<br>

**`r2ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait percentages for Round 2 NFTs.<br>


In [74]:
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

In [75]:
let processNFTTraits (nftTraits: NFTTraits) (totalNfts: float) =
    let recordType = typeof<NFTTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [76]:
let r1ScoredTraits = processNFTTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNFTTraits nftRound2Traits r2TotalNfts

### *Illustrating the Distribution of Trait Scores*: 
This section visualizes the percentages of trait occurrences alongside their corresponding scores. The createCharts function processes the trait data and generates doughnut charts that display both the raw values and their proportions.

In [78]:
let createCharts (traitScores: Map<string, Map<string, float>>) = 
    traitScores
    |> Map.fold( fun acc title labelsAndValues ->
        let labels,values = 
            labelsAndValues
            |> Map.toList
            |> List.unzip

        let combinedText = 
            List.map2 (fun label value -> sprintf "%s: %.2f" label value) labels values

        let chart = 
            Chart.Doughnut(values = values, Labels = labels, Hole = 0.5, MultiText = combinedText)
            |> Chart.withTitle title
            |> Chart.withLayout(
                Layout.init(
                    Width = 1200,
                    Height = 1000
                )
            )
                    
        chart :: acc
    )[]
    |> List.rev

let chartRound1 = createCharts r1ScoredTraits 
let chartRound2 = createCharts r2ScoredTraits

### Teddy Bear Club Round 1 Trait Scores

In [ ]:
for chart in chartRound1 do
    display(chart |> Chart.withSize(1400,1200))

### Teddy Bears Club Round 2 Trait Scores

In [ ]:
for chart in chartRound2 do
    display(chart |> Chart.withSize(1400,1200)) 

Error: input.fsx (2,27)-(2,28) parse error Missing qualification after '.'

### 2.2 Compute overall rarity ranks for each NFT. 
Using the information from this step, we also created a CSV mapping NFTs to their ranks for easier access and computation.

#### **Function**:
1. **computeNFTRank**: 
- Computes the rarity rank of NFTs based on their traits and scores.
- **Parameters**:
    - **`nftList`**: (NFT list): A list of NFTs to rank.
    - **`nftTraitScores`**: (Map<string, Map<string, float>>): Trait scores computed above.

#### **Variables**:
**`rankedNFTsRound1`**: 
- (list): Ranks Round 1 NFTs as (rank, assetName, encodedName, rarityScore).

**`rankedNFTsRound2`**: 
- (list): Ranks Round 2 NFTs with the same structure as above.


In [121]:
let computeNFTRank (nftList: list<NFT>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [122]:
let rankedNFTsRound1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTsRound2 = computeNFTRank nftRound2NFTS r2ScoredTraits

## **3. NFT Rewards Calculation** 
To calculate the Tedy rewards of a user given their owned NFTs, we first identify the number of NFTs they own, from what round they are from, and the rank of their NFTs. Then, we compute the user’s percent share of all sold NFTs based on the number of NFTs they own across all tiers. Using the percent share, we then factor in the remaining pool of available Tedy token rewards to determine the user’s share of Tedy. Next, using the number and rank of the user's owned NFTs, along with their calculated percent share, we compute the total Tedy tokens they are eligible to receive. Finally, the final Tedy reward allocation is adjusted based on the updated market cap of 8,000,000 Tedy tokens.

---

### **3.1 Count NFT Occurrences by Tier**
Determine the total number of NFTs owned in each round and categorize them based on their rarity rank within predefined ranges.

#### **Functions**:
1. **`categorizeNFTRound1ByRank`**:
- Categorizes Round 1 NFTs into predefined rank ranges, updating the counts accordingly.
- **Parameters**:
  - `nft` (string) - The identifier of the NFT.
  - `rank` (int) - The rarity rank of the NFT.
  - `nftsCount` (int array) - Array representing counts of NFTs within predefined rank ranges for Round 1.

2. **`categorizeNFTRound2ByRank`**:
- Similar to `categorizeNFTRound1ByRank` but tailored for Round 2 NFTs with different rank ranges.
- **Parameters**:
  - `nft` (string): The identifier of the NFT.
  - `rank` (int): The rarity rank of the NFT.
  - `nftsCount` (int array): Array representing counts of NFTs within predefined rank ranges for Round 2.

3. **`countNFTsPerRound`**:
- Processes a list of owned NFTs, categorizing them by their rarity rank for both Round 1 and Round 2.
- Calls categorizeNFTRound1ByRank or categorizeNFTRound2ByRank
- **Parameters**:
  - `ownedNFTs` (list<string>): A list of NFTs owned by the user.
  - `rankData` (Map<string, int>): A mapping of NFT identifiers to their rarity ranks.
- **Returns**:
  - A tuple of arrays (`round1NFTsHeldByRank`, `round2NFTsHeldByRank`): each array contains counts of NFTs in the predefined rank ranges for each round.

---

#### **Variables**:
 **`round1PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 1.

 **`round2PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 2.

 **`round1NFTsHeldByRank`**: 
   - (int array): represents counts of NFTs categorized by rank ranges for Round 1.
   - **Structure**: `[countRank1; countRank2; countRank3; countRank4; countRank5]`

 **`round2NFTsHeldByRank`**: array
   - (int array): representscounts of NFTs categorized by rank ranges for Round 2.
   - **Structure**: `[countRank1; countRank2; countRank3; countRank4; countRank5]`


In [57]:
let round1PolicyId = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87"
let round2PolicyId = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c"

In [ ]:
let categorizeNFTRound1ByRank nft rank (nftsCount: int array)=
    printfn "Calculating rewards for NFT1: %s with rank %d" nft rank
    if rank <= 15 then nftsCount[0] <- nftsCount[0] + 1
    elif rank <= 100 then nftsCount[1] <- nftsCount[1]+ 1
    elif rank <= 250 then nftsCount[2] <- nftsCount[2] + 1
    elif rank <= 500 then nftsCount[3] <- nftsCount[3] + 1
    elif rank <= 804 then nftsCount[4] <- nftsCount[4] + 1
    else printfn "Rank %d for NFT %s does not fall within any predefined range" rank nft    

In [59]:
let categorizeNFTRound2ByRank nft rank (nftsCount: int array)=
    printfn "Calculating rewards for NFT2: %s with rank %d" nft rank
    if rank <= 23 then nftsCount[0] <- nftsCount[0] + 1
    elif rank <= 221 then nftsCount[1] <- nftsCount[1] + 1
    elif rank <= 664 then nftsCount[2] <- nftsCount[2] + 1
    elif rank <= 1327 then nftsCount[3] <- nftsCount[3] + 1
    elif rank <= 2035 then nftsCount[4] <- nftsCount[4] + 1
    else printfn "Rank %d for NFT %s does not fall within any predefined range" rank nft

In [58]:
let countNFTsPerRound (ownedNFTs: list<string>) (rankData: Map<string, int>) =
    let round1NFTsHeldByRank = [|0;0;0;0;0|]
    let round2NFTsHeldByRank = [|0;0;0;0;0|]
    ownedNFTs
    |> List.iter (fun nft ->
        match rankData.TryFind nft with
        | Some rank ->
            if nft.Contains(round1PolicyId) then
                categorizeNFTRound1ByRank nft rank round1NFTsHeldByRank
            elif nft.Contains(round2PolicyId) then
                categorizeNFTRound2ByRank nft rank round2NFTsHeldByRank
            else
                printfn "Unknown NFT policyId for %s" nft 
        | None ->
            printfn "Rank data not found for NFT: %s" nft
    )
    (round1NFTsHeldByRank, round2NFTsHeldByRank)

The cell below shows test data, in order to show that we can identify the rank of a given subject.

In [ ]:
let ownedNFTs = [
                    "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c7562313738";
                    "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c54656464794265617273436c756234373930"
                ]

let round1NFTsHeldByRank, round2NFTsHeldByRank = countNFTsPerRound ownedNFTs subjectRankData

printfn "Round 1 Rewards: %A" round1NFTsHeldByRank
printfn "Round 2 Rewards: %A" round2NFTsHeldByRank

### **3.2 Calculate User's Percent Share of NFTs**
Calculate the user’s percentage share of all sold NFTs based on the number of NFTs they own across all tiers. Since Round 1 NFTs had a minting price of 350 ADA, they are given a bonus multiplier of 2.75, which increases their weight in the computations. As a result, the formulas for calculating percent share differ between the two rounds.

Where:
- Round 1 NFTs Held: The number of NFTs the user owns from Round 1.
- Round 2 NFTs Held: The number of NFTs the user owns from Round 2.
- Round 1 Bonus: The bonus multiplier for Round 1 NFTs (2.75).
- Round 1 NFTs Sold: The total number of Round 1 NFTs sold (804).
- Round 2 NFTs Sold: The total number of Round 2 NFTs sold (2035).

$\text{Round 1 Percent Share} = \frac{\text{Round 1 NFTs Held} \times \text{Round 1 Bonus}}{(\text{Round 1 NFTs Sold} \times \text{Round 1 Bonus} + \text{Round 2 NFTs Sold}) \times 100}$

$\text{Round 2 Percent Share} = \frac{\text{Round 2 NFTs Held}}{(\text{Round 1 NFTs Sold} \times \text{Round 1 Bonus} + \text{Round 2 NFTs Sold}) \times 100}$

---

#### **Functions**:
1. **`calculatePercentShareFromAvailableRewards`**:
- Computes the percent share of NFTs owned by the user for both rounds, based on the total NFTs sold and the Round 1 bonus multiplier.
- **Parameters**:
   - None explicitly passed, but it relies on:
      - `round1NFTsHeldByRank` (array) - The counts of NFTs owned by the user for Round 1.
      - `round2NFTsHeldByRank` (array) - The counts of NFTs owned by the user for Round 2.
      - `round1Bonus` (float) - The bonus multiplier for Round 1 NFTs.
      - `round1NFTsSold` (int) - Total number of NFTs sold in Round 1.
      - `round2NFTsSold` (int)  Total number of NFTs sold in Round 2.
- **Returns**:
   - A tuple of the computed percent shares for Round 1 and Round 2:
      (round1PercentShare, round2PercentShare)

---

#### **Variables**:
**`round1NFTsHeldByRank`**:
   - (int array): represents the counts of NFTs owned by the user for Round 1, categorized by rank ranges.

**`round2NFTsHeldByRank`**:
   - (int array): represents the counts of NFTs owned by the user for Round 2, categorized by rank ranges.

**`round1PercentShare`**:
   - (float): The computed percentage share of Round 1 NFTs owned by the user.

**`round2PercentShare`**:
   - (float): The computed percentage share of Round 2 NFTs owned by the user.

In [ ]:
let round1Bonus = 2.75
let round1NFTsSold = 804
let round2NFTsSold = 2035

In [ ]:
let calculatePercentShareFromAvailableRewards = 
    let round1NFTsHeld = float (Array.sum round1NFTsHeldByRank)
    let round2NFTsHeld = float (Array.sum round2NFTsHeldByRank)
    let totalNFTsSold = (float round1NFTsSold * round1Bonus + float round2NFTsSold)

    let round1PercentShare = round1NFTsHeld * round1Bonus / totalNFTsSold * 100.0
    let round2PercentShare = round2NFTsHeld / totalNFTsSold * 100.0
    (round1PercentShare,round2PercentShare)

In [62]:
let round1PercentShare, round2PercentShare = calculatePercentShareFromAvailableRewards

### **3.3 Adjust Rewards Based on Available Tedy Tokens**
Factor in the remaining pool of available Tedy token rewards to determine the user’s adjusted reward share.

Where:
- $ \text{TierTokens}_i $: The number of tokens allocated per NFT in tier $\text{( i )}$
- $ \text{countOfTier}_i $: The number of NFTs in tier $\text{( i )}$
- maxRoundTwoTedyTokens: The total number of Tedy Tokens rewards.

$\text{Distributed Tokens} = \sum_{i=1}^{5} (\text{TierTokens}_i \times \text{countOfTier}_i)$<br>
$\text{Available Token Rewards} = \text{maxRoundTwoTedyTokens} - \text{distributedTokens}$<br>
$\text{Round 1 Tedy Share} = \text{availableTokenRewards} * \text{round1PercentShare}$<br>
$\text{Round 2 Tedy Share} = \text{availableTokenRewards} * \text{round2PercentShare}$<br>

---

#### **Functions**:
1. **`calculateApproximateTbcRewards`**:
   - Calculates the approximate Tedy token rewards for Round 1 and Round 2 based on the user’s percent share and remaining available tokens.
   - **Parameters**:
     - None explicitly passed, but relies on:
       - `availableTokenRewards`: The total tokens available after subtracting distributed tokens.
       - `round1PercentShare`: The user’s percent share of Round 1 NFTs.
       - `round2PercentShare`: The user’s percent share of Round 2 NFTs.
   - **Returns**:
     - A tuple of approximate rewards: (`round1TedyShare`, `round2TedyShare`).

---

#### **Variables**:
**`round1Tier1`, `round1Tier2`, `round1Tier3`, `round1Tier4`, `round1Tier5`**:
   - Tokens allocated per NFT for each tier in Round 1:
     - **round1Tier1**: `30800`
     - **round1Tier2**: `19250`
     - **round1Tier3**: `13860`
     - **round1Tier4**: `12320`
     - **round1Tier5**: `11550`

**`round2Tier1`, `round2Tier2`, `round2Tier3`, `round2Tier4`, `round2Tier5`**:
   - Tokens allocated per NFT for each tier in Round 2:
     - **round2Tier1**: `7000`
     - **round2Tier2**: `6000`
     - **round2Tier3**: `5000`
     - **round2Tier4**: `4500`
     - **round2Tier5**: `4200`

**`countOfTier1`, `countOfTier2`, `countOfTier3`, `countOfTier4`, `countOfTier5`**:
   - The number of NFTs held in each tier:
     - **countOfTier1**: `23`
     - **countOfTier2**: `198`
     - **countOfTier3**: `443`
     - **countOfTier4**: `663`
     - **countOfTier5**: `708`

**`maxRoundTwoTedyTokens`**:
   - (int): The maximum number of Tedy tokens allocated for Round 2.
   - **Value**: `42000000`

**`distributedTokens`**:
   - (int): The total tokens distributed across all tiers for Round 2

**`availableTokenRewards`**:
   - (float): The remaining Tedy tokens available after distribution

**`round1TedyShare`, `round2TedyShare`**:
   - (int): The approximate Tedy token rewards for Round 1 and Round 2



In [63]:
let round1Tier1 = 30800
let round1Tier2 = 19250
let round1Tier3 = 13860
let round1Tier4 = 12320
let round1Tier5 = 11550

let round2Tier1 = 7000
let round2Tier2 = 6000
let round2Tier3 = 5000
let round2Tier4 = 4500
let round2Tier5 = 4200

let countOfTier1 = 23
let countOfTier2 = 198
let countOfTier3 = 443
let countOfTier4 = 663
let countOfTier5 = 708

let maxRoundTwoTedyTokens = 42000000
let distributedTokens = (countOfTier1 * round2Tier1) + (countOfTier2 * round2Tier2) + (countOfTier3 * round2Tier3) + (countOfTier4 * round2Tier4) + (countOfTier5 * round2Tier5)
let availableTokenRewards = float (maxRoundTwoTedyTokens - distributedTokens)

In [64]:
let calculateApproximateTbcRewards = 
    let round1Share = round1PercentShare / 100.0
    let round2Share = round2PercentShare / 100.0

    let round1ApproximateTedyShare = Math.Ceiling(availableTokenRewards * round1Share)
    let round2ApproximateTedyShare = Math.Ceiling(availableTokenRewards * round2Share)

    ((int)round1ApproximateTedyShare,(int)round2ApproximateTedyShare)

In [65]:
let round1TedyShare,round2TedyShare = calculateApproximateTbcRewards

### **3.4 Compute Total Tedy Allocation**
Use the number and rank of the user's owned NFTs, along with the calculated percent share above, to compute the total Tedy tokens they are eligible to receive.

Where:
- NFT Rarity Rank: The number of owned NFTs for each rank.
- Rank Tedy Value: The Tedy value assigned to the rank.

$\text{Base Tedy} = \sum_{i=1}^{5} (\text{NFT Rarity Rank Count}_i \times \text{Rank Tedy Value}_i)$<br>

$\text{Round 1 Total Tedy Allocation} = \text{Base Tedy} + \text{Round 1 Tedy Share}$

$\text{Round 2 Total Tedy Allocation} = \text{Base Tedy} + \text{Round 2 Tedy Share}$

---

### **3.4 Calculate Total Tedy Tokens**
Calculate the total Tedy tokens earned by the user in Round 1 and Round 2, combining tokens from held NFTs across tiers and additional rewards.

#### **Functions**:
1. **`calculateTedyTokensTotal`**:
- Computes the total Tedy tokens earned in Round 1 and Round 2, including tokens from held NFTs in each tier and approximate Tedy shares.
- **Parameters**:
   - None explicitly passed, but it relies on:
      - `round1NFTsHeldByRank`: An array of counts of NFTs held by the user for each tier in Round 1.
      - `round2NFTsHeldByRank`: An array of counts of NFTs held by the user for each tier in Round 2.
      - `round1TedyShare`: Approximate Tedy rewards for Round 1 based on percent share.
      - `round2TedyShare`: Approximate Tedy rewards for Round 2 based on percent share.
      - `round1Tier1`, `round1Tier2`, ..., `round2Tier5`: Tokens allocated per NFT for each tier in Round 1 and Round 2.
- **Returns**:
   - A tuple: (`round1TotalTedyTotal`, `round2TotalTedyTotal`).

---

#### **Variables**:
**`round1NFTsHeldByRank`**:
   - array of counts of NFTs held by the user for each tier in Round 1.
   - **Structure**: `[countTier1, countTier2, countTier3, countTier4, countTier5]`.

**`round2NFTsHeldByRank`**:
   - An array of counts of NFTs held by the user for each tier in Round 2.
   - **Structure**: `[countTier1, countTier2, countTier3, countTier4, countTier5]`.

**`round1TedyShare`**:
   - Approximate Tedy rewards for Round 1 based on percent share.
   - **Type**: Integer.

**`round2TedyShare`**:
   - Approximate Tedy rewards for Round 2 based on percent share.
   - **Type**: Integer.

**`round1Tier1`, `round1Tier2`, `round1Tier3`, `round1Tier4`, `round1Tier5`**:
   - Tokens allocated per NFT for each tier in Round 1.

**`round2Tier1`, `round2Tier2`, `round2Tier3`, `round2Tier4`, `round2Tier5`**:
   - Tokens allocated per NFT for each tier in Round 2.

**`round1TotalTedy`**:
   - Total Tedy tokens from held NFTs in Round 1, calculated as:

**`round2TotalTedy`**:
   - Total Tedy tokens from held NFTs in Round 2, calculated as:

**`round1TotalTedyTotal`, `round2TotalTedyTotal`**:
   - Total Tedy tokens for Round 1 and Round 2, including additional rewards:


In [ ]:
let calculateTedyTokensTotal = 
    let round1TotalTedy = (round1NFTsHeldByRank[0] * round1Tier1) + (round1NFTsHeldByRank[1] * round1Tier2) + (round1NFTsHeldByRank[2] * round1Tier3) + (round1NFTsHeldByRank[3] * round1Tier4) + (round1NFTsHeldByRank[4] * round1Tier5)
    let round2TotalTedy = (round2NFTsHeldByRank[0] * round2Tier1) + (round2NFTsHeldByRank[1] * round2Tier2) + (round2NFTsHeldByRank[2] * round2Tier3) + (round2NFTsHeldByRank[3] * round2Tier4) + (round2NFTsHeldByRank[4] * round2Tier5)

    let round1TotalTedyTotal = round1TotalTedy + round1TedyShare
    let round2TotalTedyTotal = round2TotalTedy + round2TedyShare
    (round1TotalTedyTotal,round2TotalTedyTotal)

In [ ]:
let round1ApproximateTotalTedy,round2ApproximateTotalTedy = calculateTedyTokensTotal

### **3.5 Adjust Rewards for Market Cap**
Adjust the final Tedy reward allocation based on the updated market cap of 8,000,000 to ensure alignment with the revised tokenomics.

Where:
- NFT Rarity Rank: The number of owned NFTs for each rank.
- Rank Tedy Value: The Tedy value assigned to the rank.

$\text{Final Round Reward} = (\text{Total Tedy Allocation} * \text{Current Marketcap}) / \text{Previous Market Cap}$

---

Adjust the total Tedy rewards for Round 1 and Round 2 based on the change in market capitalization. The adjustment ensures that rewards are proportional to the current market cap compared to a past market cap.

#### **Functions**:
1. **`marketcapAdjustment`**:
- Adjusts the reward amount based on the ratio of the current market cap to the past market cap.
- **Parameters**:
   - `rewards` (float): The original reward amount to be adjusted.
- **Returns**:
   - (int): The adjusted reward amount.

---

#### **Variables**:
**`pastMarketCap`**:
   - The previous market cap of the token, used as a reference for adjustments.

**`currentMarketCap`**:
   - The current market cap of the token, used to scale rewards.

**`round1ApproximateTotalTedy`**:
   - The total Tedy tokens for Round 1 before market cap adjustment.

**`round2ApproximateTotalTedy`**:
   - The total Tedy tokens for Round 2 before market cap adjustment

**`totalTedyRound1`**:
   - The adjusted total Tedy tokens for Round 1

**`totalTedyRound2`**:
   - The adjusted total Tedy tokens for Round 2

**`totalTedyReward`**:
   - The combined adjusted Tedy tokens for both rounds


In [ ]:
let pastMarketCap = 100000000.0
let currentMarketCap = 8000000.0
let marketcapAdjustment (rewards: float) = 
    let finalRoundReward = (rewards*currentMarketCap)/pastMarketCap
    Math.Ceiling(finalRoundReward)

let totalTedyRound1 = int(marketcapAdjustment round1ApproximateTotalTedy)
let totalTedyRound2 = int(marketcapAdjustment round2ApproximateTotalTedy)
let totalTedyReward = totalTedyRound1 + totalTedyRound2


4147
1092
5239


In [ ]:
printfn "%d" totalTedyRound1
printfn "%d" totalTedyRound2
printfn "%d" totalTedyReward